In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Library

In [ ]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from numpy import sort
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mtick
from wordcloud import WordCloud, STOPWORDS 
from random import sample
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, precision_recall_curve
import scipy as sp
from scipy import stats


from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras import Sequential

# Read data

In [ ]:
df_2014 = pd.read_csv("../input/200-financial-indicators-of-us-stocks-20142018/2014_Financial_Data.csv",index_col="Unnamed: 0")
df_2015 = pd.read_csv("../input/200-financial-indicators-of-us-stocks-20142018/2015_Financial_Data.csv",index_col="Unnamed: 0")
df_2016 = pd.read_csv("../input/200-financial-indicators-of-us-stocks-20142018/2016_Financial_Data.csv",index_col="Unnamed: 0")
df_2017 = pd.read_csv("../input/200-financial-indicators-of-us-stocks-20142018/2017_Financial_Data.csv",index_col="Unnamed: 0")
df_2018 = pd.read_csv("../input/200-financial-indicators-of-us-stocks-20142018/2018_Financial_Data.csv",index_col="Unnamed: 0")

# 2015

In [ ]:
data_info = pd.set_option('display.max_columns', None)
data_info=pd.DataFrame( df_2015.dtypes).T.rename(index={0:'column type'})
data_info=data_info.append(pd.DataFrame( df_2015.isnull().sum()).T.rename(index={0:'null values (nb)'}))
data_info=data_info.append(pd.DataFrame(df_2015.isnull().sum()/ df_2015.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(data_info)

In [ ]:
nas_by_feature = df_2015.isnull().sum(axis=0)
features_to_drop = nas_by_feature[nas_by_feature>200].index
df_2015.drop(features_to_drop, axis=1, inplace=True)

In [ ]:
df_2015a = df_2015[["Earnings Before Tax Margin", "Net Profit Margin", "Days Sales Outstanding",
                    "EPS", "Gross Margin", "EBIT Margin",	"EBITDA"
]]

df_2015a = df_2015a.dropna(axis=0)

In [ ]:
ss = StandardScaler()
df_2015_std = ss.fit_transform(df_2015a)
df_2015_std = ss.transform(df_2015a)

In [ ]:
# import hierarchical clustering libraries
import scipy.cluster.hierarchy as sch

# create dendrogram
dendrogram_2015 = sch.dendrogram(sch.linkage(df_2015a , method='ward'))
plt.axhline(y=3.5, color='r', linestyle='--')

In [ ]:
# import hierarchical clustering libraries
from sklearn.cluster import AgglomerativeClustering

# create clusters
hc_2015 = AgglomerativeClustering(n_clusters=5, affinity = 'euclidean', linkage = 'ward')
# save clusters for chart
y_hc_2015 = hc_2015.fit_predict(df_2015_std)

In [ ]:
#Plot a line graph to see the characteristics of the clusters
df_2015['cluster_2015'] = pd.Series(y_hc_2015, index=df_2015a.index)
data_2015_clustered_std = df_2015.groupby('cluster_2015').mean()
a = data_2015_clustered_std
a/10

In [ ]:
data_2015_clustered_std = data_2015_clustered_std[["Earnings Before Tax Margin", "Net Profit Margin",
                     "Gross Margin", "EBIT Margin",	"2016 PRICE VAR [%]", "Class"
]]

In [ ]:
#Plot a line graph to see the characteristics of the clusters
plt.plot(data_2015_clustered_std.T )
plt.xticks(rotation=90)

In [ ]:
data_2015_clustered_stda = stats.zscore(data_2015_clustered_std, axis=1, nan_policy='omit')

In [ ]:
#Plot a line graph to see the characteristics of the clusters
plt.plot(data_2015_clustered_stda.T )
plt.xticks(rotation=90)

# 2016

In [ ]:
data_info = pd.set_option('display.max_columns', None)
data_info=pd.DataFrame( df_2016.dtypes).T.rename(index={0:'column type'})
data_info=data_info.append(pd.DataFrame( df_2016.isnull().sum()).T.rename(index={0:'null values (nb)'}))
data_info=data_info.append(pd.DataFrame(df_2016.isnull().sum()/ df_2016.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(data_info)

In [ ]:
nas_by_feature = df_2016.isnull().sum(axis=0)
features_to_drop = nas_by_feature[nas_by_feature>1000].index
df_2016.drop(features_to_drop, axis=1, inplace=True)

In [ ]:
df_2016a = df_2016.dropna(axis=0)
df_2016a.drop(['Sector','2017 PRICE VAR [%]','Class'],1 , inplace=True) 

In [ ]:
ss = StandardScaler()
df_2016_std = ss.fit_transform(df_2016a)
df_2016_std = ss.transform(df_2016a)

In [ ]:
# import hierarchical clustering libraries
import scipy.cluster.hierarchy as sch

# create dendrogram
dendrogram_2016 = sch.dendrogram(sch.linkage(df_2016a , method='ward'))
plt.axhline(y=3.5, color='r', linestyle='--')

In [ ]:
# import hierarchical clustering libraries
from sklearn.cluster import AgglomerativeClustering

# create clusters
hc_2016 = AgglomerativeClustering(n_clusters=5, affinity = 'euclidean', linkage = 'ward')
# save clusters for chart
y_hc_2016 = hc_2016.fit_predict(df_2016_std)

In [ ]:
#Plot a line graph to see the characteristics of the clusters
df_2016['cluster_2016'] = pd.Series(y_hc_2016, index=df_2016a.index)
data_2016_clustered_std = df_2016.groupby('cluster_2016').mean()
a = data_2016_clustered_std
a/10

In [ ]:
data_2016_clustered_std = data_2016_clustered_std[["Earnings Before Tax Margin", "Net Profit Margin",
                     "Gross Margin", "EBIT Margin",	"2017 PRICE VAR [%]", "Class"
]]

In [ ]:
data_2016_clustered_stda = stats.zscore(data_2016_clustered_std, axis=1, nan_policy='omit')

In [ ]:
#Plot a line graph to see the characteristics of the clusters
plt.plot(data_2016_clustered_stda.T )
plt.xticks(rotation=90)